# Predict damage to buildings after earthquake in 

Predict `damage_grade`

In [38]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import f1_score, classification_report

In [24]:
X_train = pd.read_csv('data/train_values.csv').set_index('building_id')
y_train = pd.read_csv('data/train_labels.csv').set_index('building_id')

In [25]:
X_train.shape

(260601, 38)

In [26]:
X_train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [27]:
y_train.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


In [28]:
X_train.select_dtypes('object').nunique()

land_surface_condition     3
foundation_type            5
roof_type                  3
ground_floor_type          5
other_floor_type           4
position                   4
plan_configuration        10
legal_ownership_status     4
dtype: int64

# Split Data

Already have target vector and feature matrix. Use k-fold cross-validation.

# Baseline

In [29]:
print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

Baseline Accuracy: 0.5689118614280068


# Build Model

- `LogisticRegression`
-`LogisticRegressionCV`

In [32]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    LogisticRegressionCV(n_jobs=6)
)

In [33]:
model.fit(X_train, y_train);

C:\Users\dakot\anaconda3\envs\unit2\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\dakot\anaconda3\envs\unit2\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [22]:
## Logistic Regression can cover multi-class predictions with OvR
## Creates three different predictors (process of elimination)

# Check Metrics

In [37]:
print('Model Accuracy:', model.score(X_train, y_train))
print('Model F1 Score:', f1_score(y_train, model.predict(X_train), average='micro'))

Model Accuracy: 0.5923960383881873
Model F1 Score: 0.5923960383881873


In [39]:
print(classification_report(y_train, model.predict(X_train)))

              precision    recall  f1-score   support

           1       0.57      0.29      0.38     25124
           2       0.60      0.88      0.71    148259
           3       0.55      0.19      0.29     87218

    accuracy                           0.59    260601
   macro avg       0.58      0.45      0.46    260601
weighted avg       0.58      0.59      0.54    260601



# Make Prediction

In [40]:
X_test = pd.read_csv('data/test_values.csv', index_col='building_id')

In [41]:
y_pred = model.predict_proba(X_test)

In [42]:
y_pred

array([[0.0360004 , 0.5052626 , 0.458737  ],
       [0.0248195 , 0.70815863, 0.26702187],
       [0.05444693, 0.54417808, 0.40137499],
       ...,
       [0.13683748, 0.47872786, 0.38443467],
       [0.01622197, 0.64978769, 0.33399035],
       [0.21440325, 0.71374658, 0.07185017]])

In [43]:
y_pred = model.predict(X_test)

In [45]:
submission = pd.DataFrame({'damage_grade':y_pred}, index=X_test.index)

In [46]:
submission.head()

,damage_grade
building_id,
300051,2
99355,2
890251,2
745817,2
421793,3


In [47]:
submission.to_csv('submissions/2020-11-06_sumission')